# Prepare the dataset

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset('OmarAboBakr/AIC_preprocessed_dataset')



In [ ]:
# dataset['train'].to_csv('train.csv')
# dataset['validation'].to_csv('validation.csv')
# dataset['test'].to_csv('test.csv')

In [5]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train':'train.csv', 'validation':'validation.csv', 'test':'test.csv'})

Found cached dataset csv (C:/Users/Omar/.cache/huggingface/datasets/csv/default-7c4767214f85e1d6/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset["train"][0]

{'text': 'يعزا تراجع إيراد +ات ياهو ب+ شكل كبير إلى ال+ انخفاض ال+ حاد في نشاط ال+ إعلان +ات ال+ مرئي +ة ال+ رقمي +ة و+ تساعد ف+ ل+ اري مطور +ي ال+ تطبيق +ات على تحليل ال+ بيان +ات ال+ متعلق +ة ب+ مستخدمي +هم و+ توفير إعلان +ات تناسب أذواق +هم . و+ اشترى بعض منافس +ي ياهو شرك +ات تعمل في تكنولوجيا إعلان +ات ال+ هواتف ال+ محمول +ة في محاول +ة ل+ جذب ال+ مسوق +ين إلى تطبيق +ات +ها و+ تحقيق إيراد +ات من ال+ إعلان +ات على تطبيق +ات شرك +ات أخرى . و+ لم تكشف ال+ شركتان عن ال+ بنود ال+ مالي +ة ل+ صفق +ة ال+ استحواذ . لكن بعض ال+ تقارير أشار +ت إلى أن ياهو دفع +ت ما بين 200 و 300 مليون دولار ل+ ال+ استحواذ على ف+ ل+ اري ، ل+ تكون ب+ ذلك واحد +ة من أكبر عملي +ات ال+ استحواذ ل+ ياهو منذ أن استحوذ +ت على منص +ة تدوين " تومبير " عام 2012 . مواضيع قد تهم +ك نهاي +ة تعزيز ال+ إيراد +ات و+ تأتي صفق +ة ياهو ل+ ال+ استحواذ على ف+ ل+ اري بعد أيام فقط من إعلان تراجع أرباح +ها ب+ واقع 18 في ال+ مئ +ة إلى 270 مليون دولار خلال ال+ أشهر ال+ ثلاث +ة حتى نهاي +ة يونيو - حزيران ال+ ماضي . و+ تراجع +ت ال+ إيراد

In [7]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
# from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer

# Preprocess the dataset

In [ ]:
# model_name = "aubmindlab/bert-base-arabertv2"
# preprocessor = ArabertPreprocessor(model_name=model_name)

In [ ]:
# def preprocess_function(text):
#   return preprocessor.preprocess(text)

# import re

# def unpreprocess_function(text):
#   temp = preprocessor.unpreprocess(text)
#   temp = re.sub('\[CLS\]|\[PAD\]|\[SEP\]', '', temp)
#   return re.sub('\s+$|^\s', '', temp)

# Load the tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained("./AraT5")
model = AutoModelForSeq2SeqLM.from_pretrained("./AraT5")

# Encode the dataset with the tokenizer

In [9]:
import pandas as pd
import datasets
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
val_df = pd.DataFrame(dataset['validation'])
test_df = test_df.dropna()
train_df = train_df.dropna()
val_df = val_df.dropna()
train_dataset = datasets.Dataset.from_dict(train_df)
test_dataset = datasets.Dataset.from_dict(test_df)
val_dataset = datasets.Dataset.from_dict(val_df)
dataset = datasets.DatasetDict({"train":train_dataset,"test":test_dataset,"validation":val_dataset})


del train_df
del test_df
del val_df
del train_dataset
del test_dataset
del val_dataset

In [10]:
max_input_length = 1024
# max_length=1024
max_target_length = 128

# i=0

def tokenize_function(examples):
  # global i
  # i+=1
  # inputs = [preprocess_function(text) for text in examples['text']]
  
  model_inputs = tokenizer(examples['text'], padding='max_length', truncation=True)
  # print('check1', i)


  labels = tokenizer(examples['summary'], padding='max_length', max_length=128, truncation=True, return_tensors="pt")
  # print('check2', i)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs


# def tokenize_function(examples):
#   inputs = examples['text']
#   model_inputs = tokenizer(inputs, padding='max_length', truncation=True)

#   labels = tokenizer(examples['summary'], padding='max_length', max_length=128, truncation=True)

#   model_inputs['labels'] = labels['input_ids']
#   return model_inputs


In [11]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)


small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

del tokenized_dataset
del dataset

Map:   0%|          | 0/60875 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/7610 [00:00<?, ? examples/s]

Map:   0%|          | 0/7610 [00:00<?, ? examples/s]

In [ ]:
# import sys
# print(sys.getdefaultencoding())

In [12]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Load metrics

In [13]:
import evaluate
import numpy as np

rouge = evaluate.load('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# import torch
# torch.cuda.is_available()

In [14]:
import gc
gc.collect()

1180

# Train the model

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="finetuned-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True
    # fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()